# Metrics
## RMSD, p-value, INF, DI
done with RNA_assessment

In [1]:
import sys,os
# import warnings
# warnings.filterwarnings("ignore")

from RNA_assessment import RNA_normalizer

from operator import attrgetter

RESIDUES_LIST = "RNA_assessment/data/residues.list"
ATOMS_LIST = "RNA_assessment/data/atoms.list"

def CleanFormat(f):
	os.system( "mac2unix -q %s" %f )
	os.system( "dos2unix -q %s" %f )

#### Normalization

In [2]:
def normalize_structure(struct, out_file = None, index_file=None, extract_file = None):
	pdb_normalizer = RNA_normalizer.PDBNormalizer( RESIDUES_LIST, ATOMS_LIST )
	ok = pdb_normalizer.parse( struct, out_file )
	if not ok:
		sys.stderr.write("ERROR: structure not normalized!\n")
	else:
		sys.stderr.write("INFO: Normalization succeded!\n")
	if not extract_file is None:
		coords=open(index_file).read()
		RNA_normalizer.extract_PDB(SOLUTION_NORMAL,coords, extract_file)
		sys.stderr.write("INFO:	structure extracted\n")

In [3]:
from Bio.PDB.PDBParser import PDBParser

parser = PDBParser()

def create_index(file):
    structure = parser.get_structure("1", file)
    model = structure[0]
    chain = model['A']
    file_index = file.replace('.pdb', '.index')
    output_index = f'A:1:{len(chain)}'
    with open(file_index, 'w') as f:
        f.write(output_index)

    return file_index

In [4]:

# test_files = ['test/clust01_qrna.pdb', 'test/clust02_qrna.pdb']
# Normalize PDB format, correct residue names and atom names. 
# normalize_structure(test_files[0], out_file='/scr/aldea/kgutenbrunner/working/scripts/analysis/test/clust01_qrna_norm.pdb')
# normalize_structure(test_files[1], out_file='/scr/aldea/kgutenbrunner/working/scripts/analysis/test/clust02_qrna_norm.pdb')

### RMSD and P-value

In [5]:
# PVALUE set according to Hajdin et al., RNA (7) 16, 2010, either "+" or "-"
def calc_RMSD(native_file, native_index, prediction_file, prediction_index, PVALUE = "-"):
	res_struct = RNA_normalizer.PDBStruct()
	res_struct.load( native_file, native_index )
	res_raw_seq = res_struct.raw_sequence()
	
	sol_struct = RNA_normalizer.PDBStruct()
	sol_struct.load( prediction_file, prediction_index )
	sol_raw_seq = sol_struct.raw_sequence()
	
	if( sol_raw_seq != res_raw_seq ):
		sys.stderr.write("ERROR Result sequence != Solution sequence!\n")
		sys.stderr.write("DATA Solution sequence --> '%s'\n" %sol_raw_seq )
		sys.stderr.write("DATA Result sequence   --> '%s'\n" %res_raw_seq )
		return(-1)
	# computes the RMSD
	comparer = RNA_normalizer.PDBComparer()
	rmsd = comparer.rmsd( sol_struct, res_struct )
	sys.stderr.write("INFO Partial RMSD --> %f\n" %rmsd )
	pvalue = comparer.pvalue( rmsd, len(sol_raw_seq), PVALUE )
	sys.stderr.write("INFO Partial P-Value --> %e\n" %pvalue )
	return(rmsd, pvalue)

### Interaction Network Fidelity

In [6]:
def InteractionNetworkFidelity(native_file, native_index, prediction_file, prediction_index):
	res_struct = RNA_normalizer.PDBStruct()
	res_struct.load( native_file, native_index )
	res_raw_seq = res_struct.raw_sequence()
	
	sol_struct = RNA_normalizer.PDBStruct()
	sol_struct.load( prediction_file, prediction_index )
	sol_raw_seq = sol_struct.raw_sequence()
	
	if( sol_raw_seq != res_raw_seq ):
		sys.stderr.write("ERROR Result sequence != Solution sequence!\n")
		sys.stderr.write("DATA Solution sequence --> '%s'\n" %sol_raw_seq )
		sys.stderr.write("DATA Result sequence   --> '%s'\n" %res_raw_seq )
		return(-1)
	# computes the RMSD
	comparer = RNA_normalizer.PDBComparer()
	rmsd = comparer.rmsd( sol_struct, res_struct )
	INF_ALL = comparer.INF( sol_struct, res_struct, type="ALL" )
	DI_ALL = rmsd / INF_ALL
	INF_WC = comparer.INF( sol_struct, res_struct, type="PAIR_2D" )
	INF_NWC = comparer.INF( sol_struct, res_struct, type="PAIR_3D" )
	INF_STACK = comparer.INF( sol_struct, res_struct, type="STACK" )
	return (rmsd,DI_ALL, INF_ALL, INF_WC, INF_NWC,INF_STACK)

In [7]:
from Bio.PDB.PDBExceptions import PDBConstructionWarning
import warnings
warnings.filterwarnings("ignore", category=PDBConstructionWarning)

## CAD
done with voronota

In [8]:
def calculate_cad(native_file, prediction_file):
    voronota = '/scr/aldea/kgutenbrunner/opt/rna_analysis/voronota_1.28.4132/voronota-cadscore'
    voronota_result = !$voronota -m $native_file -t $prediction_file 
    voronota_result = voronota_result[0].split(' ')
    cad_score = float(voronota_result[4])
    area_target = float(voronota_result[5])
    area_model = float(voronota_result[6])
    return cad_score

### TM-Score

In [9]:
import re

def calculate_TM(native_file, prediction_file):
    TM_regexr = 'TM-score *= *(\d+.\d+)'
    GTD_TS_regexr = 'GDT-TS-score *= *(\d+.\d+)'
    TM = '/scr/aldea/kgutenbrunner/opt/rna_analysis/TMscore'
    TM_result = !$TM $native_file $prediction_file 
    for line in TM_result:
        if re.search(TM_regexr, line):
            TM_score = float(re.search(TM_regexr, line)[1])
        if re.search(GTD_TS_regexr, line):
            GTD_TS = float(re.search(GTD_TS_regexr, line)[1])
    return TM_score, GTD_TS


### $\epsilon$ RMSD
done with barnaba

In [10]:
# import barnaba
import barnaba as bb

ModuleNotFoundError: No module named 'barnaba'

In [11]:
def calculate_ermsd(native_file, prediction_file):
    return bb.ermsd(native_file,prediction_file)


## Deformation profile

In [12]:
reference_file = '/Users/katringutenbrunner/Desktop/MA/working/data/tbfv_sim/cluster1_min_std.pdb'
target_file = '/Users/katringutenbrunner/Desktop/MA/working/simRNA/04_24/QRNAs/clust02_qrna.pdb'

In [13]:
# !rna_standardize.py $target_file

In [14]:
target_file_norm = target_file.replace('.', '_std.')

In [15]:
tar2 = '/Users/katringutenbrunner/Desktop/MA/working/simRNA/03_14/QRNAs/clust01_qrna.pdb'

In [16]:
dp = '/Users/katringutenbrunner/Desktop/MA/working/opt/Deformation_Profile/dp.py'

In [17]:
!python $dp $tar2 $target_file_norm

opening reference file: '/Users/katringutenbrunner/Desktop/MA/working/simRNA/03_14/QRNAs/clust01_qrna.pdb'
/Users/katringutenbrunner/miniconda3/envs/rna_analysis/lib/python3.12/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'O' for Atom (name=OP3) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/katringutenbrunner/miniconda3/envs/rna_analysis/lib/python3.12/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'P' for Atom (name=P) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/katringutenbrunner/miniconda3/envs/rna_analysis/lib/python3.12/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'O' for Atom (name=OP1) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/katringutenbrunner/miniconda3/envs/rna_analysis/lib/python3.12/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'O' for Atom (name=OP2) with given element ''
  warnin